In [1]:
import sqlite3
import pandas as pd

Прочитаем и посмотрим на наши данные, которые нам предоставили по ссылке [link](https://drive.google.com/file/d/1NWIT8Yn-GdgpBUfFO87dnIDQgmE5nj-j/view?usp=sharing) (сами данные на github положить нельзя, а данные с гугл диска поэтому в репозитории этих файлов не будет)

In [2]:
df1 = pd.read_csv('genotyping_data/genstudio.csv')
df2 = pd.read_csv('genotyping_data/metadata.csv')

/var/folders/5l/9jtg_90x5s575m1zqn_h0kgm0000gn/T/ipykernel_3604/1102169592.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('genotyping_data/genstudio.csv')


In [3]:
df1.head()

,Unnamed: 0,SNP Name,SNP Index,SNP Aux,Sample ID,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,Position,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio
0,0,1_10573221,1,0,202341831114R01C01,[T/C],-,-,-,-,-,-,1,10573221,0.0000,0.0000,0.942,0.413,1.0000,0.4040
1,1,1_10673082,2,0,202341831114R01C01,[T/C],A,A,T,T,A,A,1,10673082,0.8272,0.8076,0.039,0.968,0.0000,0.3017
2,2,1_10723065,3,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,10723065,0.8316,0.8107,0.011,1.577,0.0000,0.0388
3,3,1_11337555,4,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,11337555,0.3781,0.7925,0.045,1.104,0.0000,0.2761
4,4,1_11407894,5,0,202341831114R01C01,[A/G],G,G,G,G,B,B,1,11407894,0.9038,0.8670,0.983,1.122,0.9994,0.0022


In [4]:
df2.head()

,Unnamed: 0,dna_chip_id,breed,sex
0,0,202290551164R09C01,Д,Хр
1,1,202341831114R02C01,Д,Хр
2,2,202341831114R03C01,Д,Хр
3,3,202341831114R04C01,Д,Хр
4,4,202290551140R01C01,Д,Хр


Создадим  database для первого df и перенесем все данные из df в 1 таблицу в этой базе данных.

In [5]:
conn = sqlite3.connect('genstudio.db')
c = conn.cursor()

Создаем таблицу `gens`, устанавливаем параметры, индексом в данном случае выступает `Unnamed_0`

In [6]:
c.execute('''CREATE TABLE IF NOT EXISTS gens ('Unnamed: 0' INTEGER PRIMARY KEY, 
                                               SNP_Name TEXT,
                                               SNP_Index INTEGER
                                               SNP_Aux INTEGER,
                                               SampleID TEXT,
                                               SNP TEXT,
                                               Allele1Top TEXT,
                                               Allele2Top TEXT,
                                               Allele1Forward TEXT,
                                               Allele2Forward TEXT,
                                               Allele1AB TEXT,
                                               Allele2AB TEXT,
                                               Chr INTEGER,
                                               Position INTEGER,
                                               GC_Score INTEGER,
                                               GT_Score INTEGER,
                                               Theta INTEGER,
                                               R INTEGER,
                                               B_Allele_Freq INTEGER
                                               Log_R_Ratio INTEGER
                                               )''')
conn.commit()

Переводим наши данные в формат sql и сразу в нашу таблицу

In [7]:
df1.to_sql('gens', conn, if_exists='replace', index = False)

2000000

Переименуем **Unnamed: 0** так как не совсем понятно изначально, что она несет

In [8]:
c.execute('''  
SELECT * FROM gens''')
c.execute("ALTER TABLE gens RENAME COLUMN 'Unnamed: 0' TO id_var")
conn.commit()

Повторяем все тоже самое со второй таблицей

In [13]:
c.execute('''CREATE TABLE IF NOT EXISTS meta ('Unnamed: 0' INTEGER PRIMARY KEY,
                                               dna_chip_id TEXT
                                               breed TEXT
                                               sex TEXT
                                               )''')
conn.commit()

In [14]:
df2.to_sql('meta', conn, if_exists='replace', index = False)

841

Мы дропаем Unnamed: 0 из нашего db, так как в данном случае он также выступает в качестве индекса. Также изменим название `dna_chip_id` на `Sample ID`, чтобы проще можно было соединять таблицы друг с другом. (Тут я немного не понял как таблицу добавить сразу в db поэтому пришлось использовать пандас)

In [15]:
c.execute('''  
SELECT * FROM meta''')
c.execute("ALTER TABLE meta RENAME COLUMN dna_chip_id TO 'Sample ID'")
c.execute("ALTER TABLE meta DROP COLUMN 'Unnamed: 0'")
conn.commit()


Соединяем оба наши таблицы по колонкам Sample_ID записав результат в переменную res

In [16]:
res = conn.execute(''' SELECT *
              FROM gens
              JOIN meta
              USING ('Sample ID')
            ''')

In [17]:
last = pd.DataFrame(res.fetchall())

In [21]:
last.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0,1_10573221,1,0,202341831114R01C01,[T/C],-,-,-,-,...,1,10573221,0.0000,0.0000,0.942,0.413,1.0000,0.4040,Д,Хр
1,1,1_10673082,2,0,202341831114R01C01,[T/C],A,A,T,T,...,1,10673082,0.8272,0.8076,0.039,0.968,0.0000,0.3017,Д,Хр
2,2,1_10723065,3,0,202341831114R01C01,[A/G],A,A,T,T,...,1,10723065,0.8316,0.8107,0.011,1.577,0.0000,0.0388,Д,Хр
3,3,1_11337555,4,0,202341831114R01C01,[A/G],A,A,T,T,...,1,11337555,0.3781,0.7925,0.045,1.104,0.0000,0.2761,Д,Хр
4,4,1_11407894,5,0,202341831114R01C01,[A/G],G,G,G,G,...,1,11407894,0.9038,0.8670,0.983,1.122,0.9994,0.0022,Д,Хр


In [23]:
c.execute('''CREATE TABLE IF NOT EXISTS genes_with_meta (ind INTEGER PRIMARY KEY, 
                                               SNP_Name TEXT,
                                               SNP_Index INTEGER
                                               SNP_Aux INTEGER,
                                               SampleID TEXT,
                                               SNP TEXT,
                                               Allele1Top TEXT,
                                               Allele2Top TEXT,
                                               Allele1Forward TEXT,
                                               Allele2Forward TEXT,
                                               Allele1AB TEXT,
                                               Allele2AB TEXT,
                                               Chr INTEGER,
                                               Position INTEGER,
                                               GC_Score INTEGER,
                                               GT_Score INTEGER,
                                               Theta INTEGER,
                                               R INTEGER,
                                               B_Allele_Freq INTEGER
                                               Log_R_Ratio INTEGER
                                               breed TEXT
                                               sex TEXT
                                               )''')
conn.commit()

In [24]:
last.to_sql('genes_with_meta', conn, if_exists='replace', index = False)

2000000

In [27]:
conn.commit()

ProgrammingError: Cannot operate on a closed database.

In [26]:
conn.close()